
<div style="text-align: center; line-height: 0; padding-top: 9px;">
  <img src="https://databricks.com/wp-content/uploads/2018/03/db-academy-rgb-1200px.png" alt="Databricks Learning">
</div>


# Lab - Creating Bronze Tables from JSON Files
### Duration: ~ 15 minutes

In this lab you will ingest a JSON file as Delta table and then flatten the JSON formatted string column.

### Learning Objectives
  - Inspect a raw JSON file.
  - Read in JSON files to a Delta table and flatten the JSON formatted string column.

## REQUIRED - SELECT CLASSIC COMPUTE

Before executing cells in this notebook, please select your classic compute cluster in the lab. Be aware that **Serverless** is enabled by default and you have a Shared SQL warehouse.

<!-- ![Select Cluster](./Includes/images/selecting_cluster_info.png) -->

Follow these steps to select the classic compute cluster:


1. Navigate to the top-right of this notebook and click the drop-down menu to select your cluster. By default, the notebook will use **Serverless**.

2. If your cluster is available, select it and continue to the next cell. If the cluster is not shown:

   - Click **More** in the drop-down.

   - In the **Attach to an existing compute resource** window, use the first drop-down to select your unique cluster.

**NOTE:** If your cluster has terminated, you might need to restart it in order to select it. To do this:

1. Right-click on **Compute** in the left navigation pane and select *Open in new tab*.

2. Find the triangle icon to the right of your compute cluster name and click it.

3. Wait a few minutes for the cluster to start.

4. Once the cluster is running, complete the steps above to select your cluster.


## A. Classroom Setup

Run the following cell to configure your working environment for this notebook.

**NOTE:** The `DA` object is only used in Databricks Academy courses and is not available outside of these courses. It will dynamically reference the information needed to run the course in the lab environment.

In [0]:
%run ./Includes/Classroom-Setup-07L

Note: you may need to restart the kernel using %restart_python or dbutils.library.restartPython() to use updated packages.


----------------------------------------------------------------------------------------
Directory /Volumes/dbacademy/ops/labuser10983516_1758981184@vocareum_com/csv_demo_files already exists. No action taken.
Directory /Volumes/dbacademy/ops/labuser10983516_1758981184@vocareum_com/json_demo_files already exists. No action taken.
Directory /Volumes/dbacademy/ops/labuser10983516_1758981184@vocareum_com/xml_demo_files already exists. No action taken.
----------------------------------------------------------------------------------------



Generated file: /Volumes/dbacademy/ops/labuser10983516_1758981184@vocareum_com/json_demo_files/lab_kafka_events.json


File written with 1 malformed 'timestamp' at position 55: /Volumes/dbacademy/ops/labuser10983516_1758981184@vocareum_com/json_demo_files/lab_kafka_events_challenge.json


Run the cell below to view your default catalog and schema. Notice that your default catalog is **dbacademy** and your default schema is your unique **labuser** schema.

**NOTE:** The default catalog and schema are pre-configured for you to avoid the need to specify the three-level name when writing your tables (i.e., catalog.schema.table).

In [0]:
SELECT current_catalog(), current_schema()

## B. Lab - JSON Ingestion
**Scenario:** You are working with your data team on ingesting a JSON file into Databricks. Your job is to ingest the JSON file as is into a bronze table, then create a second bronze table that flattens the JSON formatted string column in the raw bronze table for downstream processing.

### B1. Inspect the Dataset

1. In the cell below, view the value of the Python variable `DA.paths.working_dir`. This variable will reference your **dbacademy.ops.labuser** volume, as each user has a different source volume. This variable is created within the classroom setup script to dynamically reference your unique volume.

   Run the cell and review the results. You’ll notice that the `DA.paths.working_dir` variable points to your `/Volumes/dbacademy/ops/labuser` volume.

**Note:** Instead of using the `DA.paths.working_dir` variable, you could also specify the path name directly by right clicking on your volume and selecting **Copy volume path**.

In [0]:
%python
print(DA.paths.working_dir)

/Volumes/dbacademy/ops/labuser10983516_1758981184@vocareum_com


Run the cell to view the data in the `/Volumes/dbacademy/ops/your-labuser-name/json_demo_files/lab_kafka_events.json` file in the location from above.

In [0]:
%python
spark.sql(f'''
          SELECT * 
          FROM json.`{DA.paths.working_dir}/json_demo_files/lab_kafka_events.json`
          ''').display()

key,timestamp,value
event_0,1.758982315453146E9,eyJ1c2VyX2lkIjogInVzZXJfMzM4NSIsICJldmVudF90eXBlIjogInZpZXciLCAiZXZlbnRfdGltZXN0YW1wIjogIjIwMjUtMDktMjdUMTQ6MTE6NTUuNDUzMDk2IiwgIml0ZW1zIjogW3siaXRlbV9pZCI6ICJpdGVtXzIzNCIsICJxdWFudGl0eSI6IDMsICJwcmljZV91c2QiOiAyOS4yM31dfQ==
event_1,1.758982315453181E9,eyJ1c2VyX2lkIjogInVzZXJfMTU0OSIsICJldmVudF90eXBlIjogInB1cmNoYXNlIiwgImV2ZW50X3RpbWVzdGFtcCI6ICIyMDI1LTA5LTI3VDE0OjExOjU1LjQ1MzE1NSIsICJpdGVtcyI6IFt7Iml0ZW1faWQiOiAiaXRlbV80NTYiLCAicXVhbnRpdHkiOiAyLCAicHJpY2VfdXNkIjogMjYuMDV9XX0=
event_2,1.758982315453219E9,eyJ1c2VyX2lkIjogInVzZXJfMTgyMiIsICJldmVudF90eXBlIjogInB1cmNoYXNlIiwgImV2ZW50X3RpbWVzdGFtcCI6ICIyMDI1LTA5LTI3VDE0OjExOjU1LjQ1MzE4NiIsICJpdGVtcyI6IFt7Iml0ZW1faWQiOiAiaXRlbV8yMjYiLCAicXVhbnRpdHkiOiAxLCAicHJpY2VfdXNkIjogNTAuNDF9LCB7Iml0ZW1faWQiOiAiaXRlbV81MTciLCAicXVhbnRpdHkiOiAyLCAicHJpY2VfdXNkIjogODEuNTR9LCB7Iml0ZW1faWQiOiAiaXRlbV82NDAiLCAicXVhbnRpdHkiOiA0LCAicHJpY2VfdXNkIjogNjguMjF9XX0=
event_3,1.758982315453253E9,eyJ1c2VyX2lkIjogInVzZXJfNjc1OSIsICJldmVudF90eXBlIjogInB1cmNoYXNlIiwgImV2ZW50X3RpbWVzdGFtcCI6ICIyMDI1LTA5LTI3VDE0OjExOjU1LjQ1MzIyMyIsICJpdGVtcyI6IFt7Iml0ZW1faWQiOiAiaXRlbV81MzIiLCAicXVhbnRpdHkiOiAzLCAicHJpY2VfdXNkIjogMTMuNTZ9LCB7Iml0ZW1faWQiOiAiaXRlbV8yMjQiLCAicXVhbnRpdHkiOiAzLCAicHJpY2VfdXNkIjogMzkuNTF9LCB7Iml0ZW1faWQiOiAiaXRlbV8yMDIiLCAicXVhbnRpdHkiOiAzLCAicHJpY2VfdXNkIjogODYuMDd9XX0=
event_4,1.75898231545328E9,eyJ1c2VyX2lkIjogInVzZXJfMzMzNCIsICJldmVudF90eXBlIjogImNsaWNrIiwgImV2ZW50X3RpbWVzdGFtcCI6ICIyMDI1LTA5LTI3VDE0OjExOjU1LjQ1MzI1NyIsICJpdGVtcyI6IFt7Iml0ZW1faWQiOiAiaXRlbV8yNzMiLCAicXVhbnRpdHkiOiAxLCAicHJpY2VfdXNkIjogMzIuNDN9LCB7Iml0ZW1faWQiOiAiaXRlbV84MTEiLCAicXVhbnRpdHkiOiAyLCAicHJpY2VfdXNkIjogNjIuNDh9XX0=
event_5,1.75898231545332E9,eyJ1c2VyX2lkIjogInVzZXJfOTI0MSIsICJldmVudF90eXBlIjogInZpZXciLCAiZXZlbnRfdGltZXN0YW1wIjogIjIwMjUtMDktMjdUMTQ6MTE6NTUuNDUzMjg1IiwgIml0ZW1zIjogW3siaXRlbV9pZCI6ICJpdGVtXzU4NSIsICJxdWFudGl0eSI6IDEsICJwcmljZV91c2QiOiA4OC44NH0sIHsiaXRlbV9pZCI6ICJpdGVtXzcxOSIsICJxdWFudGl0eSI6IDIsICJwcmljZV91c2QiOiA0My42OX0sIHsiaXRlbV9pZCI6ICJpdGVtXzQwNSIsICJxdWFudGl0eSI6IDEsICJwcmljZV91c2QiOiA0MS4yM31dfQ==
event_6,1.758982315453352E9,eyJ1c2VyX2lkIjogInVzZXJfNTA4MCIsICJldmVudF90eXBlIjogInZpZXciLCAiZXZlbnRfdGltZXN0YW1wIjogIjIwMjUtMDktMjdUMTQ6MTE6NTUuNDUzMzI1IiwgIml0ZW1zIjogW3siaXRlbV9pZCI6ICJpdGVtXzczMSIsICJxdWFudGl0eSI6IDIsICJwcmljZV91c2QiOiA1OS41OH0sIHsiaXRlbV9pZCI6ICJpdGVtXzk5OSIsICJxdWFudGl0eSI6IDMsICJwcmljZV91c2QiOiAzNi41M30sIHsiaXRlbV9pZCI6ICJpdGVtXzY1MCIsICJxdWFudGl0eSI6IDIsICJwcmljZV91c2QiOiA1OC4xOH1dfQ==
event_7,1.758982315453384E9,eyJ1c2VyX2lkIjogInVzZXJfNTE4MyIsICJldmVudF90eXBlIjogImNsaWNrIiwgImV2ZW50X3RpbWVzdGFtcCI6ICIyMDI1LTA5LTI3VDE0OjExOjU1LjQ1MzM1NiIsICJpdGVtcyI6IFt7Iml0ZW1faWQiOiAiaXRlbV81NjciLCAicXVhbnRpdHkiOiA1LCAicHJpY2VfdXNkIjogNzMuMjZ9LCB7Iml0ZW1faWQiOiAiaXRlbV85MDAiLCAicXVhbnRpdHkiOiAzLCAicHJpY2VfdXNkIjogMzMuMjV9LCB7Iml0ZW1faWQiOiAiaXRlbV8zNzkiLCAicXVhbnRpdHkiOiAxLCAicHJpY2VfdXNkIjogNzEuNDN9XX0=
event_8,1.758982315453411E9,eyJ1c2VyX2lkIjogInVzZXJfMTQwOSIsICJldmVudF90eXBlIjogImNsaWNrIiwgImV2ZW50X3RpbWVzdGFtcCI6ICIyMDI1LTA5LTI3VDE0OjExOjU1LjQ1MzM4OCIsICJpdGVtcyI6IFt7Iml0ZW1faWQiOiAiaXRlbV85NDMiLCAicXVhbnRpdHkiOiAxLCAicHJpY2VfdXNkIjogNjQuMzV9LCB7Iml0ZW1faWQiOiAiaXRlbV80MTIiLCAicXVhbnRpdHkiOiA0LCAicHJpY2VfdXNkIjogMzYuMjF9XX0=
event_9,1.758982315453461E9,eyJ1c2VyX2lkIjogInVzZXJfMjE2MSIsICJldmVudF90eXBlIjogInB1cmNoYXNlIiwgImV2ZW50X3RpbWVzdGFtcCI6ICIyMDI1LTA5LTI3VDE0OjExOjU1LjQ1MzQzNyIsICJpdGVtcyI6IFt7Iml0ZW1faWQiOiAiaXRlbV83NjYiLCAicXVhbnRpdHkiOiAyLCAicHJpY2VfdXNkIjogMTYuODh9LCB7Iml0ZW1faWQiOiAiaXRlbV85NDAiLCAicXVhbnRpdHkiOiAzLCAicHJpY2VfdXNkIjogNjQuNDZ9XX0=


### B2. Create the Raw Bronze Table

Inspect and run the code below to ingest the raw JSON file `/Volumes/dbacademy/ops/your-labuser-name/json_demo_files/lab_kafka_events.json` and create the **lab7_lab_kafka_events_raw** table.

Notice the following:
- The **value** column is decoded.
- The **decoded_value** column was created and returns the decoded column as a JSON-formatted string.


In [0]:
CREATE OR REPLACE TABLE lab7_lab_kafka_events_raw
AS
SELECT 
  *,
  cast(unbase64(value) as STRING) as decoded_value
FROM read_files(
        DA.paths_working_dir || '/json_demo_files/lab_kafka_events.json',
        format => "json", 
        schema => '''
          key STRING, 
          timestamp DOUBLE, 
          value STRING
        ''',
        rescueddatacolumn => '_rescued_data'
      );

-- View the table
SELECT *
FROM lab7_lab_kafka_events_raw;

key,timestamp,value,_rescued_data,decoded_value
event_0,1.758982315453146E9,eyJ1c2VyX2lkIjogInVzZXJfMzM4NSIsICJldmVudF90eXBlIjogInZpZXciLCAiZXZlbnRfdGltZXN0YW1wIjogIjIwMjUtMDktMjdUMTQ6MTE6NTUuNDUzMDk2IiwgIml0ZW1zIjogW3siaXRlbV9pZCI6ICJpdGVtXzIzNCIsICJxdWFudGl0eSI6IDMsICJwcmljZV91c2QiOiAyOS4yM31dfQ==,null,"{""user_id"": ""user_3385"", ""event_type"": ""view"", ""event_timestamp"": ""2025-09-27T14:11:55.453096"", ""items"": [{""item_id"": ""item_234"", ""quantity"": 3, ""price_usd"": 29.23}]}"
event_1,1.758982315453181E9,eyJ1c2VyX2lkIjogInVzZXJfMTU0OSIsICJldmVudF90eXBlIjogInB1cmNoYXNlIiwgImV2ZW50X3RpbWVzdGFtcCI6ICIyMDI1LTA5LTI3VDE0OjExOjU1LjQ1MzE1NSIsICJpdGVtcyI6IFt7Iml0ZW1faWQiOiAiaXRlbV80NTYiLCAicXVhbnRpdHkiOiAyLCAicHJpY2VfdXNkIjogMjYuMDV9XX0=,null,"{""user_id"": ""user_1549"", ""event_type"": ""purchase"", ""event_timestamp"": ""2025-09-27T14:11:55.453155"", ""items"": [{""item_id"": ""item_456"", ""quantity"": 2, ""price_usd"": 26.05}]}"
event_2,1.758982315453219E9,eyJ1c2VyX2lkIjogInVzZXJfMTgyMiIsICJldmVudF90eXBlIjogInB1cmNoYXNlIiwgImV2ZW50X3RpbWVzdGFtcCI6ICIyMDI1LTA5LTI3VDE0OjExOjU1LjQ1MzE4NiIsICJpdGVtcyI6IFt7Iml0ZW1faWQiOiAiaXRlbV8yMjYiLCAicXVhbnRpdHkiOiAxLCAicHJpY2VfdXNkIjogNTAuNDF9LCB7Iml0ZW1faWQiOiAiaXRlbV81MTciLCAicXVhbnRpdHkiOiAyLCAicHJpY2VfdXNkIjogODEuNTR9LCB7Iml0ZW1faWQiOiAiaXRlbV82NDAiLCAicXVhbnRpdHkiOiA0LCAicHJpY2VfdXNkIjogNjguMjF9XX0=,null,"{""user_id"": ""user_1822"", ""event_type"": ""purchase"", ""event_timestamp"": ""2025-09-27T14:11:55.453186"", ""items"": [{""item_id"": ""item_226"", ""quantity"": 1, ""price_usd"": 50.41}, {""item_id"": ""item_517"", ""quantity"": 2, ""price_usd"": 81.54}, {""item_id"": ""item_640"", ""quantity"": 4, ""price_usd"": 68.21}]}"
event_3,1.758982315453253E9,eyJ1c2VyX2lkIjogInVzZXJfNjc1OSIsICJldmVudF90eXBlIjogInB1cmNoYXNlIiwgImV2ZW50X3RpbWVzdGFtcCI6ICIyMDI1LTA5LTI3VDE0OjExOjU1LjQ1MzIyMyIsICJpdGVtcyI6IFt7Iml0ZW1faWQiOiAiaXRlbV81MzIiLCAicXVhbnRpdHkiOiAzLCAicHJpY2VfdXNkIjogMTMuNTZ9LCB7Iml0ZW1faWQiOiAiaXRlbV8yMjQiLCAicXVhbnRpdHkiOiAzLCAicHJpY2VfdXNkIjogMzkuNTF9LCB7Iml0ZW1faWQiOiAiaXRlbV8yMDIiLCAicXVhbnRpdHkiOiAzLCAicHJpY2VfdXNkIjogODYuMDd9XX0=,null,"{""user_id"": ""user_6759"", ""event_type"": ""purchase"", ""event_timestamp"": ""2025-09-27T14:11:55.453223"", ""items"": [{""item_id"": ""item_532"", ""quantity"": 3, ""price_usd"": 13.56}, {""item_id"": ""item_224"", ""quantity"": 3, ""price_usd"": 39.51}, {""item_id"": ""item_202"", ""quantity"": 3, ""price_usd"": 86.07}]}"
event_4,1.75898231545328E9,eyJ1c2VyX2lkIjogInVzZXJfMzMzNCIsICJldmVudF90eXBlIjogImNsaWNrIiwgImV2ZW50X3RpbWVzdGFtcCI6ICIyMDI1LTA5LTI3VDE0OjExOjU1LjQ1MzI1NyIsICJpdGVtcyI6IFt7Iml0ZW1faWQiOiAiaXRlbV8yNzMiLCAicXVhbnRpdHkiOiAxLCAicHJpY2VfdXNkIjogMzIuNDN9LCB7Iml0ZW1faWQiOiAiaXRlbV84MTEiLCAicXVhbnRpdHkiOiAyLCAicHJpY2VfdXNkIjogNjIuNDh9XX0=,null,"{""user_id"": ""user_3334"", ""event_type"": ""click"", ""event_timestamp"": ""2025-09-27T14:11:55.453257"", ""items"": [{""item_id"": ""item_273"", ""quantity"": 1, ""price_usd"": 32.43}, {""item_id"": ""item_811"", ""quantity"": 2, ""price_usd"": 62.48}]}"
event_5,1.75898231545332E9,eyJ1c2VyX2lkIjogInVzZXJfOTI0MSIsICJldmVudF90eXBlIjogInZpZXciLCAiZXZlbnRfdGltZXN0YW1wIjogIjIwMjUtMDktMjdUMTQ6MTE6NTUuNDUzMjg1IiwgIml0ZW1zIjogW3siaXRlbV9pZCI6ICJpdGVtXzU4NSIsICJxdWFudGl0eSI6IDEsICJwcmljZV91c2QiOiA4OC44NH0sIHsiaXRlbV9pZCI6ICJpdGVtXzcxOSIsICJxdWFudGl0eSI6IDIsICJwcmljZV91c2QiOiA0My42OX0sIHsiaXRlbV9pZCI6ICJpdGVtXzQwNSIsICJxdWFudGl0eSI6IDEsICJwcmljZV91c2QiOiA0MS4yM31dfQ==,null,"{""user_id"": ""user_9241"", ""event_type"": ""view"", ""event_timestamp"": ""2025-09-27T14:11:55.453285"", ""items"": [{""item_id"": ""item_585"", ""quantity"": 1, ""price_usd"": 88.84}, {""item_id"": ""item_719"", ""quantity"": 2, ""price_usd"": 43.69}, {""item_id"": ""item_405"", ""quantity"": 1, ""price_usd"": 41.23}]}"
event_6,1.758982315453352E9,eyJ1c2VyX2lkIjogInVzZXJfNTA4MCIsICJldmVudF90eXBlIjogInZpZXciLCAiZXZlbnRfdGltZXN0YW1wIjogIjIwMjUtMDktMjdUMTQ6MTE6NTUuNDUzMzI1IiwgIml0ZW1zIjogW3siaXRlbV9pZCI6ICJpdGVtXzczMSI

### B3. Create the Flattened Bronze Table

1. Your goal is to flatten the JSON formatted string column **decoded_value** from the table **lab7_lab_kafka_events_raw** to create a new table named **lab7_lab_kafka_events_flattened** for downstream processing. The table should contain the following columns:
    - **key**
    - **timestamp**
    - **user_id**
    - **event_type**
    - **event_timestamp**
    - **items**

    You can use whichever technique you prefer:

    - Parse the JSON formatted string (easiest) to flatten
      - [Query JSON strings](https://docs.databricks.com/aws/en/semi-structured/json):

    - Convert the JSON formatted string as a VARIANT and flatten
      - [parse_json function](https://docs.databricks.com/gcp/en/sql/language-manual/functions/parse_json)

    - Convert the JSON formatted string to a STRUCT and flatten
      - [schema_of_json function](https://docs.databricks.com/aws/en/sql/language-manual/functions/schema_of_json)
      - [from_json function](https://docs.databricks.com/gcp/en/sql/language-manual/functions/from_json)

**NOTE:** View the lab solution notebook to view the solutions for each.

2. To begin, run the code below to view the final solution table **lab7_lab_kafka_events_flattened_solution**. This will give you an idea of what your final table should look like.

  **NOTE**: Depending on your solution, the data types of the columns may vary slightly.  


##### Optional Challenge

  As a challenge, after flattening the table, try converting the data types accordingly. Depending on your skill set, you may not convert all columns to the correct data types within the allotted time.

  - **key** STRING
  - **timestamp** DOUBLE
  - **user_id** STRING
  - **event_type** STRING
  - **event_timestamp** TIMESTAMP
  - **items** (STRUCT or VARIANT) depending on the method you used.

In [0]:
SELECT *
FROM lab7_lab_kafka_events_flattened_solution

key,timestamp,user_id,event_type,event_timestamp,items
event_0,1.758982315453146E9,user_3385,view,2025-09-27T14:11:55.453096Z,"List(List(item_234, 29.23, 3))"
event_1,1.758982315453181E9,user_1549,purchase,2025-09-27T14:11:55.453155Z,"List(List(item_456, 26.05, 2))"
event_2,1.758982315453219E9,user_1822,purchase,2025-09-27T14:11:55.453186Z,"List(List(item_226, 50.41, 1), List(item_517, 81.54, 2), List(item_640, 68.21, 4))"
event_3,1.758982315453253E9,user_6759,purchase,2025-09-27T14:11:55.453223Z,"List(List(item_532, 13.56, 3), List(item_224, 39.51, 3), List(item_202, 86.07, 3))"
event_4,1.75898231545328E9,user_3334,click,2025-09-27T14:11:55.453257Z,"List(List(item_273, 32.43, 1), List(item_811, 62.48, 2))"
event_5,1.75898231545332E9,user_9241,view,2025-09-27T14:11:55.453285Z,"List(List(item_585, 88.84, 1), List(item_719, 43.69, 2), List(item_405, 41.23, 1))"
event_6,1.758982315453352E9,user_5080,view,2025-09-27T14:11:55.453325Z,"List(List(item_731, 59.58, 2), List(item_999, 36.53, 3), List(item_650, 58.18, 2))"
event_7,1.758982315453384E9,user_5183,click,2025-09-27T14:11:55.453356Z,"List(List(item_567, 73.26, 5), List(item_900, 33.25, 3), List(item_379, 71.43, 1))"
event_8,1.758982315453411E9,user_1409,click,2025-09-27T14:11:55.453388Z,"List(List(item_943, 64.35, 1), List(item_412, 36.21, 4))"
event_9,1.758982315453461E9,user_2161,purchase,2025-09-27T14:11:55.453437Z,"List(List(item_766, 16.88, 2), List(item_940, 64.46, 3))"


3. Write the query in the cell below to read the **lab_kafka_events_raw** table and create the flattened table **lab7_lab_kafka_events_flattened** following the requirements from above.

In [0]:
---- Parse the JSON formatted STRING
CREATE OR REPLACE TABLE lab7_lab_kafka_events_flattened_str
AS
SELECT 
  key,
  timestamp,
  decoded_value:user_id,
  decoded_value:event_type,
  cast(decoded_value:event_timestamp AS TIMESTAMP),
  from_json(decoded_value:items,'ARRAY<STRUCT<item_id: STRING, price_usd: DOUBLE, quantity: BIGINT>>') AS items
FROM lab7_lab_kafka_events_raw;


---- Display the table
SELECT *
FROM lab7_lab_kafka_events_flattened_str;

key,timestamp,user_id,event_type,event_timestamp,items
event_0,1.758982315453146E9,user_3385,view,2025-09-27T14:11:55.453096Z,"List(List(item_234, 29.23, 3))"
event_1,1.758982315453181E9,user_1549,purchase,2025-09-27T14:11:55.453155Z,"List(List(item_456, 26.05, 2))"
event_2,1.758982315453219E9,user_1822,purchase,2025-09-27T14:11:55.453186Z,"List(List(item_226, 50.41, 1), List(item_517, 81.54, 2), List(item_640, 68.21, 4))"
event_3,1.758982315453253E9,user_6759,purchase,2025-09-27T14:11:55.453223Z,"List(List(item_532, 13.56, 3), List(item_224, 39.51, 3), List(item_202, 86.07, 3))"
event_4,1.75898231545328E9,user_3334,click,2025-09-27T14:11:55.453257Z,"List(List(item_273, 32.43, 1), List(item_811, 62.48, 2))"
event_5,1.75898231545332E9,user_9241,view,2025-09-27T14:11:55.453285Z,"List(List(item_585, 88.84, 1), List(item_719, 43.69, 2), List(item_405, 41.23, 1))"
event_6,1.758982315453352E9,user_5080,view,2025-09-27T14:11:55.453325Z,"List(List(item_731, 59.58, 2), List(item_999, 36.53, 3), List(item_650, 58.18, 2))"
event_7,1.758982315453384E9,user_5183,click,2025-09-27T14:11:55.453356Z,"List(List(item_567, 73.26, 5), List(item_900, 33.25, 3), List(item_379, 71.43, 1))"
event_8,1.758982315453411E9,user_1409,click,2025-09-27T14:11:55.453388Z,"List(List(item_943, 64.35, 1), List(item_412, 36.21, 4))"
event_9,1.758982315453461E9,user_2161,purchase,2025-09-27T14:11:55.453437Z,"List(List(item_766, 16.88, 2), List(item_940, 64.46, 3))"


In [0]:
---- Convert the JSON formatted string as a VARIANT
---- NOTE: The VARIANT decoded_value_variant column is included in this solution to display the column
---- NOTE: Variant data type will not work on Serverless Version 1.
CREATE OR REPLACE TABLE lab7_lab_kafka_events_flattened_variant
AS
SELECT
  key,
  timestamp,
  parse_json(decoded_value) AS decoded_value_variant,
  cast(decoded_value_variant:user_id AS STRING),
  decoded_value_variant:event_type :: STRING,
  decoded_value_variant:event_timestamp :: TIMESTAMP,
  decoded_value_variant:items
FROM lab7_lab_kafka_events_raw;


---- Display the table
SELECT *
FROM lab7_lab_kafka_events_flattened_variant;

key,timestamp,decoded_value_variant,user_id,event_type,event_timestamp,items
event_0,1.758982315453146E9,"{""event_timestamp"":""2025-09-27T14:11:55.453096"",""event_type"":""view"",""items"":[{""item_id"":""item_234"",""price_usd"":29.23,""quantity"":3}],""user_id"":""user_3385""}",user_3385,view,2025-09-27T14:11:55.453096Z,"[{""item_id"":""item_234"",""price_usd"":29.23,""quantity"":3}]"
event_1,1.758982315453181E9,"{""event_timestamp"":""2025-09-27T14:11:55.453155"",""event_type"":""purchase"",""items"":[{""item_id"":""item_456"",""price_usd"":26.05,""quantity"":2}],""user_id"":""user_1549""}",user_1549,purchase,2025-09-27T14:11:55.453155Z,"[{""item_id"":""item_456"",""price_usd"":26.05,""quantity"":2}]"
event_2,1.758982315453219E9,"{""event_timestamp"":""2025-09-27T14:11:55.453186"",""event_type"":""purchase"",""items"":[{""item_id"":""item_226"",""price_usd"":50.41,""quantity"":1},{""item_id"":""item_517"",""price_usd"":81.54,""quantity"":2},{""item_id"":""item_640"",""price_usd"":68.21,""quantity"":4}],""user_id"":""user_1822""}",user_1822,purchase,2025-09-27T14:11:55.453186Z,"[{""item_id"":""item_226"",""price_usd"":50.41,""quantity"":1},{""item_id"":""item_517"",""price_usd"":81.54,""quantity"":2},{""item_id"":""item_640"",""price_usd"":68.21,""quantity"":4}]"
event_3,1.758982315453253E9,"{""event_timestamp"":""2025-09-27T14:11:55.453223"",""event_type"":""purchase"",""items"":[{""item_id"":""item_532"",""price_usd"":13.56,""quantity"":3},{""item_id"":""item_224"",""price_usd"":39.51,""quantity"":3},{""item_id"":""item_202"",""price_usd"":86.07,""quantity"":3}],""user_id"":""user_6759""}",user_6759,purchase,2025-09-27T14:11:55.453223Z,"[{""item_id"":""item_532"",""price_usd"":13.56,""quantity"":3},{""item_id"":""item_224"",""price_usd"":39.51,""quantity"":3},{""item_id"":""item_202"",""price_usd"":86.07,""quantity"":3}]"
event_4,1.75898231545328E9,"{""event_timestamp"":""2025-09-27T14:11:55.453257"",""event_type"":""click"",""items"":[{""item_id"":""item_273"",""price_usd"":32.43,""quantity"":1},{""item_id"":""item_811"",""price_usd"":62.48,""quantity"":2}],""user_id"":""user_3334""}",user_3334,click,2025-09-27T14:11:55.453257Z,"[{""item_id"":""item_273"",""price_usd"":32.43,""quantity"":1},{""item_id"":""item_811"",""price_usd"":62.48,""quantity"":2}]"
event_5,1.75898231545332E9,"{""event_timestamp"":""2025-09-27T14:11:55.453285"",""event_type"":""view"",""items"":[{""item_id"":""item_585"",""price_usd"":88.84,""quantity"":1},{""item_id"":""item_719"",""price_usd"":43.69,""quantity"":2},{""item_id"":""item_405"",""price_usd"":41.23,""quantity"":1}],""user_id"":""user_9241""}",user_9241,view,2025-09-27T14:11:55.453285Z,"[{""item_id"":""item_585"",""price_usd"":88.84,""quantity"":1},{""item_id"":""item_719"",""price_usd"":43.69,""quantity"":2},{""item_id"":""item_405"",""price_usd"":41.23,""quantity"":1}]"
event_6,1.758982315453352E9,"{""event_timestamp"":""2025-09-27T14:11:55.453325"",""event_type"":""view"",""items"":[{""item_id"":""item_731"",""price_usd"":59.58,""quantity"":2},{""item_id"":""item_999"",""price_usd"":36.53,""quantity"":3},{""item_id"":""item_650"",""price_usd"":58.18,""quantity"":2}],""user_id"":""user_5080""}",user_5080,view,2025-09-27T14:11:55.453325Z,"[{""item_id"":""item_731"",""price_usd"":59.58,""quantity"":2},{""item_id"":""item_999"",""price_usd"":36.53,""quantity"":3},{""item_id"":""item_650"",""price_usd"":58.18,""quantity"":2}]"
event_7,1.758982315453384E9,"{""event_timestamp"":""2025-09-27T14:11:55.453356"",""event_type"":""click"",""items"":[{""item_id"":""item_567"",""price_usd"":73.26,""quantity"":5},{""item_id"":""item_900"",""price_usd"":33.25,""quantity"":3},{""item_id"":""item_379"",""price_usd"":71.43,""quantity"":1}],""user_id"":""user_5183""}",user_5183,click,2025-09-27T14:11:55.453356Z,"[{""item_id"":""item_567"",""price_usd"":73.26,""quantity"":5},{""item_id"":""item_900"",""price_usd"":33.25,""quantity"":3},{""item_id"":""item_379"",""price_usd"":71.43,""quantity"":1}]"
event

In [0]:
---- Convert the JSON formatted string as a STRUCT

---- Return the structure of the JSON formatted string
SELECT schema_of_json(decoded_value)
FROM lab7_lab_kafka_events_raw
LIMIT 1;


---- Use the JSON structure from above within the from_json function to convert the JSON formatted string to a STRUCT
---- NOTE: The STRUCT decoded_value_struct column is included in this solution to display the column
CREATE OR REPLACE TABLE lab7_lab_kafka_events_flattened_struct
AS
SELECT
  key,
  timestamp,
  from_json(decoded_value, 'STRUCT<event_timestamp: STRING, event_type: STRING, items: ARRAY<STRUCT<item_id: STRING, price_usd: DOUBLE, quantity: BIGINT>>, user_id: STRING>') AS decoded_value_struct,
  decoded_value_struct.user_id,
  decoded_value_struct.event_type,
  cast(decoded_value_struct.event_timestamp AS TIMESTAMP),
  decoded_value_struct.items
FROM lab7_lab_kafka_events_raw;


---- Display the table
SELECT *
FROM lab7_lab_kafka_events_flattened_struct;

key,timestamp,decoded_value_struct,user_id,event_type,event_timestamp,items
event_0,1.758982315453146E9,"List(2025-09-27T14:11:55.453096, view, List(List(item_234, 29.23, 3)), user_3385)",user_3385,view,2025-09-27T14:11:55.453096Z,"List(List(item_234, 29.23, 3))"
event_1,1.758982315453181E9,"List(2025-09-27T14:11:55.453155, purchase, List(List(item_456, 26.05, 2)), user_1549)",user_1549,purchase,2025-09-27T14:11:55.453155Z,"List(List(item_456, 26.05, 2))"
event_2,1.758982315453219E9,"List(2025-09-27T14:11:55.453186, purchase, List(List(item_226, 50.41, 1), List(item_517, 81.54, 2), List(item_640, 68.21, 4)), user_1822)",user_1822,purchase,2025-09-27T14:11:55.453186Z,"List(List(item_226, 50.41, 1), List(item_517, 81.54, 2), List(item_640, 68.21, 4))"
event_3,1.758982315453253E9,"List(2025-09-27T14:11:55.453223, purchase, List(List(item_532, 13.56, 3), List(item_224, 39.51, 3), List(item_202, 86.07, 3)), user_6759)",user_6759,purchase,2025-09-27T14:11:55.453223Z,"List(List(item_532, 13.56, 3), List(item_224, 39.51, 3), List(item_202, 86.07, 3))"
event_4,1.75898231545328E9,"List(2025-09-27T14:11:55.453257, click, List(List(item_273, 32.43, 1), List(item_811, 62.48, 2)), user_3334)",user_3334,click,2025-09-27T14:11:55.453257Z,"List(List(item_273, 32.43, 1), List(item_811, 62.48, 2))"
event_5,1.75898231545332E9,"List(2025-09-27T14:11:55.453285, view, List(List(item_585, 88.84, 1), List(item_719, 43.69, 2), List(item_405, 41.23, 1)), user_9241)",user_9241,view,2025-09-27T14:11:55.453285Z,"List(List(item_585, 88.84, 1), List(item_719, 43.69, 2), List(item_405, 41.23, 1))"
event_6,1.758982315453352E9,"List(2025-09-27T14:11:55.453325, view, List(List(item_731, 59.58, 2), List(item_999, 36.53, 3), List(item_650, 58.18, 2)), user_5080)",user_5080,view,2025-09-27T14:11:55.453325Z,"List(List(item_731, 59.58, 2), List(item_999, 36.53, 3), List(item_650, 58.18, 2))"
event_7,1.758982315453384E9,"List(2025-09-27T14:11:55.453356, click, List(List(item_567, 73.26, 5), List(item_900, 33.25, 3), List(item_379, 71.43, 1)), user_5183)",user_5183,click,2025-09-27T14:11:55.453356Z,"List(List(item_567, 73.26, 5), List(item_900, 33.25, 3), List(item_379, 71.43, 1))"
event_8,1.758982315453411E9,"List(2025-09-27T14:11:55.453388, click, List(List(item_943, 64.35, 1), List(item_412, 36.21, 4)), user_1409)",user_1409,click,2025-09-27T14:11:55.453388Z,"List(List(item_943, 64.35, 1), List(item_412, 36.21, 4))"
event_9,1.758982315453461E9,"List(2025-09-27T14:11:55.453437, purchase, List(List(item_766, 16.88, 2), List(item_940, 64.46, 3)), user_2161)",user_2161,purchase,2025-09-27T14:11:55.453437Z,"List(List(item_766, 16.88, 2), List(item_940, 64.46, 3))"



&copy; 2025 Databricks, Inc. All rights reserved. Apache, Apache Spark, Spark, the Spark Logo, Apache Iceberg, Iceberg, and the Apache Iceberg logo are trademarks of the <a href="https://www.apache.org/" target="blank">Apache Software Foundation</a>.<br/>
<br/><a href="https://databricks.com/privacy-policy" target="blank">Privacy Policy</a> | 
<a href="https://databricks.com/terms-of-use" target="blank">Terms of Use</a> | 
<a href="https://help.databricks.com/" target="blank">Support</a>